In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
wins = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\data\cash hands wins.csv')
flush = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\data\flush and above.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Admin\\Desktop\\9stacks\\data\\cash hands wins.csv'

In [ ]:
cash_games = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\players cash daily.csv')
cash_winnings = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\player cash winnings daily.csv')
cashout = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\cashout daily.csv')
deposit = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\deposit daily.csv')
tourney_games = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\players tourney daily.csv')
users = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\users july-Oct.csv')
users2 = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\users feb-nov.csv')
freeroll_hands = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\freeroll hands.csv')
tourney_hands = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\tournament hands.csv')
cash_wages = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\cash wagering.csv')
user_points = pd.read_csv(r'C:\Users\Admin\Desktop\9stacks\data\loyalty points.csv')

In [ ]:
# Removing bloacked users
users['created'] = pd.to_datetime(users['created'],format = '%d-%m-%Y %H:%M')
users['created'] = users['created'].dt.date
users2['created'] = pd.to_datetime(users2['created'],format = '%d-%m-%Y %H:%M')
users2['created'] = users2['created'].dt.date
users_a = pd.merge(users,users2, on = ['player_id','created'],how = 'inner').drop(columns = ['user','Unnamed: 0'])
users_a = users_a.loc[users_a['isBlocked']==False].drop(columns = ['isBlocked'])

#Cash games
cash_games['date'] = pd.to_datetime(cash_games['date'],format = '%Y-%m-%d')
cash_games = pd.merge(cash_games,users_a,left_on = 'f_player_id',right_on =  'player_id',how  ='inner')
cash_games = cash_games.drop(columns = ['player_id','_id'])
cash_games['date'] = cash_games['date'].dt.date
cash_games['days_from_register'] = (cash_games['date']- cash_games['created']).apply(lambda x:x.days)

#cash wages
cash_wages['date'] = pd.to_datetime(cash_wages['date'],format = '%Y-%m-%d')
cash_wages = pd.merge(cash_wages,users_a,left_on = 'f_player_id',right_on =  'player_id',how  ='inner')
cash_wages = cash_wages.drop(columns = ['player_id','_id'])
cash_wages['date'] = cash_wages['date'].dt.date
cash_wages['days_from_register'] = (cash_wages['date']- cash_wages['created']).apply(lambda x:x.days)

#tourney games
tourney_games['date'] = pd.to_datetime(tourney_games['date'],format = '%Y-%m-%d')
tourney_games = pd.merge(tourney_games,users_a,left_on = 'f_player_id',right_on =  'player_id',how  ='inner')
tourney_games = tourney_games.drop(columns = ['player_id','_id'])
tourney_games['date'] = tourney_games['date'].dt.date
tourney_games['days_from_register'] = (tourney_games['date']- tourney_games['created']).apply(lambda x:x.days)
tourney_games['tourney_winnings'] = tourney_games['prize']- tourney_games['buyin']

#tourney_hands
tourney_hands['date'] = pd.to_datetime(tourney_hands['date'],format = '%Y-%m-%d')
tourney_hands = pd.merge(tourney_hands,users_a,left_on = 'f_player_id',right_on =  'player_id',how  ='inner')
tourney_hands = tourney_hands.drop(columns = ['player_id','_id'])
tourney_hands['date'] = tourney_hands['date'].dt.date
tourney_hands['days_from_register'] = (tourney_hands['date']- tourney_hands['created']).apply(lambda x:x.days)

#freeroll_hands
freeroll_hands['date'] = pd.to_datetime(freeroll_hands['date'],format = '%Y-%m-%d')
freeroll_hands = pd.merge(freeroll_hands,users_a,left_on = 'f_player_id',right_on =  'player_id',how  ='inner')
freeroll_hands = freeroll_hands.drop(columns = ['player_id','_id'])
freeroll_hands['date'] = freeroll_hands['date'].dt.date
freeroll_hands['days_from_register'] = (freeroll_hands['date']- freeroll_hands['created']).apply(lambda x:x.days)

#cash_winnings
cash_winnings['date'] = pd.to_datetime(cash_winnings['date'],format = '%Y-%m-%d')
cash_winnings = pd.merge(cash_winnings,users_a,left_on = 'f_player_id',right_on='player_id',how = 'inner').drop(columns =['_id','player_id'])
cash_winnings['date'] = cash_winnings['date'].dt.date
cash_winnings['days_from_register'] = (cash_winnings['date']- cash_winnings['created']).apply(lambda x:x.days)

#cashout
cashout['date'] = pd.to_datetime(cashout['date'],format = '%Y-%m-%d')
cashout = pd.merge(cashout,users_a,left_on = 'user',right_on = '_id',how = 'inner').drop(columns = ['user','_id'])
cashout['date'] = cashout['date'].dt.date
cashout['days_from_register'] = (cashout['date']- cashout['created']).apply(lambda x:x.days)

#deposit
deposit['date'] = pd.to_datetime(deposit['date'],format = '%Y-%m-%d')
deposit = pd.merge(deposit,users,left_on = 'user',right_on = '_id',how = 'inner').drop(columns = ['user','_id'])
deposit['date'] = deposit['date'].dt.date
deposit['days_from_register'] = (deposit['date']- deposit['created']).apply(lambda x:x.days)
deposit1 = deposit.loc[(deposit['days_from_register']>=0) & (deposit['days_from_register']<=20)]
deposit1 = deposit1.sort_values(by = ['player_id','days_from_register'])

#transactions
transactions = pd.merge(deposit,cashout, on = ['player_id','date','created','days_from_register'],how= 'outer',suffixes = ('_deposit','_cashout'))

In [ ]:
c = pd.DataFrame(cash_games['days_from_register'].value_counts())[:10]
fig = px.bar(x = c.index,y = c['days_from_register'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Number of Cash games')
fig.show()

c = pd.DataFrame(cash_games.groupby('days_from_register')['rake'].mean())[:20]
fig = px.bar(x = c.index,y = c['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Rake')
fig.show()

In [ ]:
# Users who left in the first 10 days
c = pd.DataFrame(cash_games.groupby('f_player_id')['days_from_register'].max())
end_before_10 = np.array(c.loc[c['days_from_register']<10].index)
w = []
for i in cash_games.index:
    if cash_games.loc[i,'f_player_id'] in end_before_10:
        w.append(i)
end_before_10 = cash_games.loc[w,:].sort_values(by = ['f_player_id','days_from_register'])

# Users who left in the first 3 days
end_before_3= np.array(c.loc[c['days_from_register']<3].index)
w = []
for i in cash_games.index:
    if cash_games.loc[i,'f_player_id'] in end_before_3:
        w.append(i)
end_before_3= cash_games.loc[w,:].sort_values(by = ['f_player_id','days_from_register'])

In [ ]:
#Last activity dataframe : d dataframe will have the last activity(cash games,tournaments,transactions)day for each user
# -1 indicates user didnt do that particular activity
a = set(cash_games['f_player_id'].unique())
b = set(tourney_games['f_player_id'].unique())
c = set(transactions['player_id'].unique())
e = set(freeroll_hands['f_player_id'].unique())
d = pd.DataFrame(a.union(b).union(c).union(e),columns = ['f_player_id'])
d['cash_game_last'] = -1
d['tourney_game_last'] = -1
d['deposit_last'] = -1
d['cashout_last'] = -1
d['freeroll_last'] = -1

c = pd.DataFrame(cash_games.groupby('f_player_id')['days_from_register'].max())
for i in d.index:
    if d.loc[i,'f_player_id'] in c.index:
        d.loc[i,'cash_game_last'] = c.loc[d.loc[i,'f_player_id'],'days_from_register']

c = pd.DataFrame(tourney_games.groupby('f_player_id')['days_from_register'].max())
for i in d.index:
    if d.loc[i,'f_player_id'] in c.index:
        d.loc[i,'tourney_game_last'] = c.loc[d.loc[i,'f_player_id'],'days_from_register']
        
c = pd.DataFrame(cashout.groupby('player_id')['days_from_register'].max())
for i in d.index:
    if d.loc[i,'f_player_id'] in c.index:
        d.loc[i,'cashout_last'] = c.loc[d.loc[i,'f_player_id'],'days_from_register']     
        
c = pd.DataFrame(deposit.groupby('player_id')['days_from_register'].max())
for i in d.index:
    if d.loc[i,'f_player_id'] in c.index:
        d.loc[i,'deposit_last'] = c.loc[d.loc[i,'f_player_id'],'days_from_register']

c = pd.DataFrame(freeroll_hands.groupby('f_player_id')['days_from_register'].max())
for i in d.index:
    if d.loc[i,'f_player_id'] in c.index:
        d.loc[i,'freeroll_last'] = c.loc[d.loc[i,'f_player_id'],'days_from_register']

        
d['last_activity'] = d[['cash_game_last','tourney_game_last','deposit_last','cashout_last','freeroll_last']].max(axis=1)
g = d.loc[d['last_activity']<0].index
d = d.drop(index = g)

In [ ]:
# Number of users who left per day in the first 15 days
a = pd.DataFrame(d['last_activity'].value_counts()).sort_index()[:15]
fig = go.Figure(data = go.Bar(x = a.index,y = a['last_activity'],text = a['last_activity'],textposition = 'outside'))
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Last Activity',title=  'Number of users who left in the first 15 days')

In [ ]:
# Number of users still playing after each day
t = pd.DataFrame(np.arange(0,100),columns = ['days_from_registration'])
t['users'] = 0
t.loc[0,'users'] = 7800
for i in range(1,t.shape[0]):
    t.loc[i,'users'] = t.loc[i-1,'users'] - len(d.loc[d['last_activity']==(i-1)])

fig =  go.Figure(go.Bar(x = t['days_from_registration'],y = t['users']))
fig.update_layout(title  = 'Total users after each day from their registration')

In [ ]:
# Average cash games played by each user in the first month
w = []
e = []
for i in np.arange(1,30):
    w.append(len(cash_games.loc[cash_games['days_from_register']<i])+len(tourney_games.loc[tourney_games['days_from_register']<i])+ len(freeroll_hands.loc[freeroll_hands['days_from_register']<i]))
for i in w:
    e.append(round(i/7719,2))
    
fig =  go.Figure(go.Bar(x = np.arange(1,30),y = e))
fig.update_layout(title  = 'Average number of games played per user till each day for the first month ')

In [ ]:
# Average winnings(cash+tournament) of users who left at those days
a = pd.DataFrame(cash_winnings.groupby('f_player_id')['cash_winning'].mean())
b = pd.DataFrame(tourney_games.groupby('f_player_id')['tourney_winnings'].mean())
c = pd.merge(a,b,on = 'f_player_id')
c['total_winnings'] = c['cash_winning']+ c['tourney_winnings']

c1 = c.loc[(c['total_winnings']<750) & (c['total_winnings']>-750)]
p = pd.DataFrame(np.arange(0,31,1),columns = ['days'])
p['Average Winnings'] = 0
for i in np.arange(0,30):
    k = np.array(d.loc[(d['last_activity']<p.loc[i+1,'days']) & (d['last_activity']>=p.loc[i,'days']),'f_player_id'])
    w = []
    for j in c.index:
        if j in k:
            w.append(j)
    p.loc[i,'Average Winnings'] = c.loc[w,'total_winnings'].mean()

fig =  go.Figure(go.Bar(x = p['days'],y = p['Average Winnings']))
fig.update_layout(title  = 'Average winnings of users who left after a particular day')

In [ ]:
#Collating all the data from different dataframes

collated = pd.merge(cash_games,tourney_games,on = ['f_player_id','date','created','days_from_register'],how = 'outer')
collated = pd.merge(collated,users,left_on = ['f_player_id','created'],right_on = ['player_id','created'],how = 'inner')
collated = pd.merge(collated,transactions,left_on = ['f_player_id','date','created','days_from_register'],right_on = ['player_id','date','created','days_from_register'],how = 'outer')
cash_games['avg_rake'] = cash_games['rake']* cash_games['hands']
cash_games['avg_big_blind'] = cash_games['f_big_blind']*cash_games['hands']
cash_games2 = cash_games.groupby(['f_player_id','date','created','days_from_register'],as_index = False)[['avg_rake','avg_big_blind','hands']].mean()
tourney_games2 = tourney_games.groupby(['f_player_id','date','created','days_from_register'],as_index = False)['tourney_winnings'].mean()
cash_winnings2 = cash_winnings.groupby(['f_player_id','date','created','days_from_register'],as_index = False)['cash_winning'].mean()
transactions = transactions.fillna(0)
transactions['total_deposit'] = transactions['amount_deposit'] * transactions['count_deposit']
transactions['total_cashout'] = transactions['amount_cashout'] * transactions['count_cashout']
transactions2 = transactions.groupby(['player_id','date','created','days_from_register'],as_index = False)['total_deposit','total_cashout'].mean()
collate = pd.merge(cash_games2,tourney_games2, on = ['f_player_id','date','created','days_from_register'],how  ='outer')
collate = pd.merge(collate,cash_winnings2,on =['f_player_id','date','created','days_from_register'],how  ='outer')
h = pd.merge(collate,transactions,left_on = ['f_player_id','date','created','days_from_register'],right_on = ['player_id','date','created','days_from_register'],how  ='outer')
h.loc[h['f_player_id'].isna()==True ,'f_player_id']= h.loc[h['f_player_id'].isna()==True,'player_id']
collate = h.drop(columns =  ['player_id'])
collate = pd.merge(collate,users,left_on = ['f_player_id','created'],right_on = ['player_id','created'],how = 'inner').drop(columns=  ['player_id'])
collate = collate[['f_player_id','_id','created','date','days_from_register','avg_rake','avg_big_blind','hands','cash_winning','tourney_winnings','amount_deposit','count_deposit','amount_cashout','count_cashout']]

In [ ]:
# Average cash hands per day in the first month
a = pd.DataFrame(cash_games.groupby('days_from_register')['hands'].mean())[:30]
fig = go.Figure(data = go.Bar(x = a.index,y = a['hands']))
fig.update_layout(xaxis_title = 'Days from Registration',yaxis_title= 'Average Cash hands',title=  'Average cash hands per day by user')

In [ ]:
# Average non-freeroll tournament hands each day in the first month
a = pd.DataFrame(tourney_hands.groupby('days_from_register')['hands'].mean())[:30]
fig = go.Figure(data = go.Bar(x = a.index,y = a['hands']))
fig.update_layout(xaxis_title = 'Days from Registration',yaxis_title= 'Average Non-freeroll Tournament hands',title=  'Average Tournament hands per day by user')

In [ ]:
# Average Freeroll hands each day in the first month
a = pd.DataFrame(freeroll_hands.groupby('days_from_register')['hands'].mean())[:30]
fig = go.Figure(data = go.Bar(x = a.index,y = a['hands']))
fig.update_layout(xaxis_title = 'Days from Registration',yaxis_title= 'Average Freeroll hands',title=  'Average Freeroll hands per day by user')

In [ ]:
# Average hands(total) played eah day in the first month
a = pd.DataFrame(cash_games.groupby('days_from_register')['hands'].mean())[:90]
b = pd.DataFrame(tourney_hands.groupby('days_from_register')['hands'].mean())[:90]
c = pd.DataFrame(freeroll_hands.groupby('days_from_register')['hands'].mean())[:90]

fig = go.Figure(go.Bar(x=np.arange(0,91), y = a['hands'], name='Cash Game Hands'))
fig.add_trace(go.Bar(x= np.arange(0,91), y= b['hands'], name='Tournament hands'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'},
                 xaxis_title = 'Days from Registration',yaxis_title = 'Average hands',
                 title = 'Average hands for Cash and Tournament games')
fig.show()

In [ ]:
# Average tournament (freeroll+ non-freeroll) hands played in the first month
a = pd.DataFrame(cash_games.groupby('days_from_register')['hands'].mean())[:30]
b = pd.DataFrame(tourney_hands.groupby('days_from_register')['hands'].mean())[:30]
c = pd.DataFrame(freeroll_hands.groupby('days_from_register')['hands'].mean())[:30]

fig = go.Figure(go.Bar(x=np.arange(0,31), y = b['hands']-c['hands'], name='Cash Tournament Hands'))
fig.add_trace(go.Bar(x= np.arange(0,31), y=b['hands'], name='Freeroll Tournament hands'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'},
                 xaxis_title = 'Days from Registration',yaxis_title = 'Average hands',
                 title = 'Average hands for different tournament games')
fig.show()

In [ ]:
#Average non-freeroll tournaments played per day by users
a = pd.DataFrame(tourney_games.groupby('days_from_register')['no_of_tournaments'].mean())[1:30]
fig = go.Figure(data = go.Bar(x = a.index,y = a['no_of_tournaments']))
fig.update_layout(xaxis_title = 'Days from Registration',yaxis_title= 'Average Tournament games',title=  'Average Tournament games per day by user')

### Making the model to define and test accuracy of our bucket system

In [ ]:
#final dataframe will tell us about activity(cash wagering,tourney winnings,deposit and total hands played) of user in the first 3 days

a = pd.DataFrame(cash_wages.loc[cash_wages['days_from_register']<3].groupby('f_player_id')['wager'].sum())
b = pd.DataFrame(tourney_games.loc[tourney_games['days_from_register']<3].groupby('f_player_id')['buyin'].sum())
c_hands =  pd.DataFrame(cash_games.loc[cash_games['days_from_register']<3].groupby('f_player_id')['hands'].sum())
t_hands =  pd.DataFrame(tourney_hands.loc[tourney_hands['days_from_register']<3].groupby('f_player_id')['hands'].sum())
f_hands =  pd.DataFrame(freeroll_hands.loc[freeroll_hands['days_from_register']<3].groupby('f_player_id')['hands'].sum())
h =  pd.merge(c_hands,t_hands,on = ['f_player_id'],how = 'outer',suffixes = ('_cash','_tourney'))
d = pd.merge(h,f_hands,on = 'f_player_id',how = 'outer').fillna(0)
d['total'] = d['hands_cash']+d['hands_tourney'] + d['hands']
d.reset_index(inplace=True)

c = pd.DataFrame(deposit.loc[deposit['days_from_register']<3].groupby('player_id')['amount'].sum())
c.reset_index(inplace=True)

final = pd.merge(a,b,on = 'f_player_id',how = 'outer')
final.reset_index(inplace=True)
final = pd.merge(final,c, left_on ='f_player_id',right_on = 'player_id',how = 'outer')
final['f_player_id'].fillna(final['player_id'],inplace=True)
final = final.drop(columns=['player_id'])
final = pd.merge(final,d[['f_player_id','total']],on = 'f_player_id',how = 'outer').fillna(0)

In [ ]:
# Weightage system and cuts decided upon
dep = [120,900]
hands = [250,950]
cash_wage = [400,1200]
buyin = [12,500]
weights = [0.3,0.1,0.3,0.3]
points = [-1,1.25,2]
score_limits = [-0.7,0.4]

In [ ]:
score = pd.DataFrame(final['f_player_id'])
score['deposit'] = 0
score['hands'] = 0
score['cash_wagering'] = 0
score['buyin'] = 0
score['total_score']=0

for i in range(score.shape[0]):
    if final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'amount'][i]<dep[0]:
        score.loc[i,'deposit'] = points[0]
    elif (final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'amount'][i]>=dep[0]) and final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'amount'][i]<dep[1]:
        score.loc[i,'deposit'] = points[1]
    elif final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'amount'][i]>=dep[1]:
        score.loc[i,'deposit'] = points[2]
        
    if final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'total'][i]<hands[0]:
        score.loc[i,'hands'] = points[0]
    elif (final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'total'][i]>=hands[0]) and final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'total'][i]<hands[1]:
        score.loc[i,'hands'] = points[1]
    elif final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'total'][i]>=hands[1]:
        score.loc[i,'hands'] = points[2]
        
    if final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'wager'][i]<cash_wage[0]:
        score.loc[i,'cash_wagering'] = points[0]
    elif (final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'wager'][i]>=cash_wage[0]) and final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'wager'][i]<cash_wage[1]:
        score.loc[i,'cash_wagering'] = points[1]
    elif final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'wager'][i]>=cash_wage[1]:
        score.loc[i,'cash_wagering'] = points[2]
        
    if final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'buyin'][i]<buyin[0]:
        score.loc[i,'buyin'] = points[0]
    elif (final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'buyin'][i]>=buyin[0]) and final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'buyin'][i]<buyin[1]:
        score.loc[i,'buyin'] = points[1]
    elif final.loc[final['f_player_id']==score.loc[i,'f_player_id'],'buyin'][i]>=buyin[1]:
        score.loc[i,'buyin'] = points[2]
            
    score['total_score'][i] = np.multiply(list(score.loc[i,['deposit','hands','cash_wagering','buyin']]),weights).sum()
    
    if score.loc[i,'total_score']<score_limits[0]:
        score.loc[i,'bucket']=1
    elif (score.loc[i,'total_score']>=score_limits[0]) and (score.loc[i,'total_score']<score_limits[1]):
        score.loc[i,'bucket']=2
    elif score.loc[i,'total_score']>=score_limits[1]:
        score.loc[i,'bucket']=3

In [ ]:
# Original Bucketing based on whole user life
# Our system will try to predict as close to these values as possible

bucket2 = pd.DataFrame(final['f_player_id'])
h = pd.DataFrame(tourney_games[['f_player_id','buyin']].groupby('f_player_id')['buyin'].sum())
g = pd.DataFrame(cash_wages.groupby('f_player_id')['wager'].sum())
bucket2 = pd.merge(bucket2,g, on='f_player_id',how = 'left')
bucket2 = pd.merge(bucket2,h,on = 'f_player_id',how = 'left')
bucket2['total'] = bucket2['wager']+bucket2['buyin']
bucket2.fillna(0,inplace=True)

bucket2['bucket']=0
for i in range(bucket2.shape[0]):
    if bucket2.loc[i,'total']<=300:
        bucket2.loc[i,'bucket'] = 1
    elif (bucket2.loc[i,'total']>40) and (bucket2.loc[i,'total']<=4300):
        bucket2.loc[i,'bucket'] = 2
    else:
        bucket2.loc[i,'bucket']=3

In [ ]:
score['bucket'].value_counts()

In [ ]:
bucket2['bucket'].value_counts()

In [ ]:
print(len(set(np.array(score.loc[score['bucket']==1,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==1,'f_player_id']))))*100/score['bucket'].value_counts()[1])
print(len(set(np.array(score.loc[score['bucket']==2,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==2,'f_player_id']))))*100/score['bucket'].value_counts()[2])
print(len(set(np.array(score.loc[score['bucket']==3,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==3,'f_player_id']))))*100/score['bucket'].value_counts()[3])

In [ ]:
(len(set(np.array(score.loc[score['bucket']==1,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==1,'f_player_id']))))+len(set(np.array(score.loc[score['bucket']==2,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==2,'f_player_id']))))+len(set(np.array(score.loc[score['bucket']==3,'f_player_id'])).intersection(set(np.array(bucket2.loc[bucket2['bucket']==3,'f_player_id'])))))/(score.shape[0])

In [ ]:
p =pd.merge(final,score[['f_player_id','bucket']],on = 'f_player_id',how = 'left')

In [ ]:
# Bucket 1 graphs
i = 1
q = pd.DataFrame(score.loc[score['bucket']==i,'f_player_id'])
w = pd.merge(q,deposit[['amount','player_id','days_from_register']],left_on = 'f_player_id',right_on = 'player_id',how = 'left')
w.drop(columns = ['player_id'],inplace=True)
w['days_from_register'].fillna(-1,inplace=True)
w['amount'].fillna(0,inplace=True)

import plotly.express as px
import plotly.graph_objects as go
c = pd.DataFrame(w.groupby('days_from_register')['amount'].mean()[0:10])
fig = px.bar(x = c.index,y = c['amount'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Amount Deposited')
fig.show()

w = pd.merge(q,cash_wages[['f_player_id','wager','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['wager'].mean()[0:10])
fig = px.bar(x = c.index,y = c['wager'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Wagering')
fig.show()

w = pd.merge(q,tourney_games[['f_player_id','buyin','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['buyin'].mean()[0:10])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Buyin')
fig.show()

In [ ]:
#Bucket 2 graphs
i = 2
q = pd.DataFrame(score.loc[score['bucket']==i,'f_player_id'])
w = pd.merge(q,deposit[['amount','player_id','days_from_register']],left_on = 'f_player_id',right_on = 'player_id',how = 'left')
w.drop(columns = ['player_id'],inplace=True)
w['days_from_register'].fillna(-1,inplace=True)
w['amount'].fillna(0,inplace=True)

import plotly.express as px
import plotly.graph_objects as go
c = pd.DataFrame(w.groupby('days_from_register')['amount'].mean()[0:10])
fig = px.bar(x = c.index,y = c['amount'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Amount Deposited')
fig.show()

w = pd.merge(q,cash_wages[['f_player_id','wager','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['wager'].mean()[0:10])
fig = px.bar(x = c.index,y = c['wager'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Wagering')
fig.show()

w = pd.merge(q,tourney_games[['f_player_id','buyin','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['buyin'].mean()[0:10])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Buyin')
fig.show()

In [ ]:
# Bucket3 graphs
i = 3
q = pd.DataFrame(score.loc[score['bucket']==i,'f_player_id'])
w = pd.merge(q,deposit[['amount','player_id','days_from_register']],left_on = 'f_player_id',right_on = 'player_id',how = 'left')
w.drop(columns = ['player_id'],inplace=True)
w['days_from_register'].fillna(-1,inplace=True)
w['amount'].fillna(0,inplace=True)

import plotly.express as px
import plotly.graph_objects as go
c = pd.DataFrame(w.groupby('days_from_register')['amount'].mean()[0:10])
fig = px.bar(x = c.index,y = c['amount'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Amount Deposited')
fig.show()

w = pd.merge(q,cash_wages[['f_player_id','wager','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['wager'].mean()[0:10])
fig = px.bar(x = c.index,y = c['wager'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Wagering')
fig.show()

w = pd.merge(q,tourney_games[['f_player_id','buyin','days_from_register']],on = 'f_player_id',how = 'inner')
c = pd.DataFrame(w.groupby('days_from_register')['buyin'].mean()[0:10])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Buyin')
fig.show()

In [ ]:
deposit.loc[deposit['player_id']==142586396]

### New updated bucket system

In [ ]:
cash_games

In [7]:
cash_wages

NameError: name 'cash_wages' is not defined

In [ ]:
a = pd.DataFrame(cash_games.loc[cash_games['days_from_register']<3].groupby('f_player_id')['rake'].sum())
b = pd.DataFrame(tourney_games.loc[tourney_games['days_from_register']<3].groupby('f_player_id')['buyin'].sum())
c = pd.DataFrame(cash_wages.loc[cash_wages['days_from_register']<3].groupby('f_player_id')['wager'].sum())

final = pd.merge(a,b,on = 'f_player_id',how = 'outer')
c.reset_index(inplace=True)
final.reset_index(inplace=True)
final = pd.merge(final,c,left_on = 'f_player_id',right_on= 'f_player_id',how = 'outer')
final.reset_index(inplace=True)
final.fillna(0,inplace=True)

In [ ]:
final['perc'] = final['rake']/final['wager']

In [ ]:
final['perc'].describe()

In [ ]:
final['bucket'] = 0

for i in final.index:
    if final.loc[i,'rake']>=75:
        final.loc[i,'bucket']='3c'
    elif (final.loc[i,'rake']>=15):
        final.loc[i,'bucket'] = '2c'
    elif final.loc[i,'buyin']>=300:
        final.loc[i,'bucket'] = '3t'
    elif final.loc[i,'buyin']>=50:
        final.loc[i,'bucket'] = '2t'
    else:
        final.loc[i,'bucket'] = '1'

In [ ]:
final['bucket'].value_counts()

In [ ]:
k.loc[k['currentTier']=='Gold','bucket'].value_counts()

In [ ]:
k = pd.merge(final,user3[['player_id','currentTier']],left_on = 'f_player_id',right_on = 'player_id',how = 'left')
k.loc[(k['currentTier']=='Gold') & (k['bucket']=='2c')]

In [ ]:
k.loc[(k['currentTier']=='Gold') & (k['bucket']=='1')]

In [ ]:
k.loc[(k['currentTier']=='Silver'),'bucket'].value_counts()

In [ ]:
import plotly.graph_objects as go

labels = ['1','2_Cash','3_Cash','2_Tourney','3_Tourney']
values = np.array(final['bucket'].value_counts())

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()


In [ ]:
final

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
i = '1'

a = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[0:10])
fig = px.bar(x = a.index,y = a['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Hands',title = 'Cash hands for Bucket '+i+ ' users')
fig.show()

b = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','f_big_blind']],on = 'f_player_id',how = 'left').groupby('days_from_register')['f_big_blind'].mean()[0:10])
fig = px.bar(x = b.index,y = b['f_big_blind'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Big Blind',title = 'Average Big Blinds for Bucket '+i+ ' users')
fig.show()

c = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_games[['f_player_id','days_from_register','buyin']],on = 'f_player_id',how = 'left').groupby('days_from_register')['buyin'].mean()[1:11])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Buyin',title = 'Tournament Buyin for Bucket '+i+ ' users')
fig.show()

d = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_hands[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[:10])
fig = px.bar(x = d.index,y = d['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Hands',title = 'Tournament Hands for Bucket '+i+ ' users')
fig.show()

e = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','rake']],on = 'f_player_id',how = 'left').groupby('days_from_register')['rake'].mean()[:10])
fig = px.bar(x = e.index,y = e['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Rake')
fig.show()

In [ ]:
i = '2t'

a = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[0:10])
fig = px.bar(x = a.index,y = a['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Hands',title = 'Cash hands for Bucket '+i+ ' users')
fig.show()

b = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','f_big_blind']],on = 'f_player_id',how = 'left').groupby('days_from_register')['f_big_blind'].mean()[0:10])
fig = px.bar(x = b.index,y = b['f_big_blind'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Big Blind',title = 'Average Big Blinds for Bucket '+i+ ' users')
fig.show()

c = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_games[['f_player_id','days_from_register','buyin']],on = 'f_player_id',how = 'left').groupby('days_from_register')['buyin'].mean()[1:11])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Buyin',title = 'Tournament Buyin for Bucket '+i+ ' users')
fig.show()

d = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_hands[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[:10])
fig = px.bar(x = d.index,y = d['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Hands')
fig.show()

e = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','rake']],on = 'f_player_id',how = 'left').groupby('days_from_register')['rake'].mean()[:10])
fig = px.bar(x = e.index,y = e['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Rake',title = 'Cash games rake for Bucket '+i+ ' users')
fig.show()

In [ ]:
i = '2c'

a = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[0:10])
fig = px.bar(x = a.index,y = a['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Hands',title = 'Cash hands for Bucket '+i+ ' users')
fig.show()

b = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','f_big_blind']],on = 'f_player_id',how = 'left').groupby('days_from_register')['f_big_blind'].mean()[0:10])
fig = px.bar(x = b.index,y = b['f_big_blind'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Big Blind',title = 'Average Big Blinds for Bucket '+i+ ' users')
fig.show()

c = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_games[['f_player_id','days_from_register','buyin']],on = 'f_player_id',how = 'left').groupby('days_from_register')['buyin'].mean()[1:11])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Buyin',title = 'Tournament Buyin for Bucket '+i+ ' users')
fig.show()

d = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_hands[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[:10])
fig = px.bar(x = d.index,y = d['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Hands')
fig.show()

e = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','rake']],on = 'f_player_id',how = 'left').groupby('days_from_register')['rake'].mean()[:10])
fig = px.bar(x = e.index,y = e['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Rake',title = 'Cash games rake for Bucket '+i+ ' users')
fig.show()

In [ ]:
i = '3t'

a = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[0:10])
fig = px.bar(x = a.index,y = a['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Hands',title = 'Cash hands for Bucket '+i+ ' users')
fig.show()

b = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','f_big_blind']],on = 'f_player_id',how = 'left').groupby('days_from_register')['f_big_blind'].mean()[0:10])
fig = px.bar(x = b.index,y = b['f_big_blind'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Big Blind',title = 'Average Big Blinds for Bucket '+i+ ' users')
fig.show()

c = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_games[['f_player_id','days_from_register','buyin']],on = 'f_player_id',how = 'left').groupby('days_from_register')['buyin'].mean()[1:11])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Buyin',title = 'Tournament Buyin for Bucket '+i+ ' users')
fig.show()

d = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_hands[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[:10])
fig = px.bar(x = d.index,y = d['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Hands')
fig.show()

e = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','rake']],on = 'f_player_id',how = 'left').groupby('days_from_register')['rake'].mean()[:10])
fig = px.bar(x = e.index,y = e['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Rake',title = 'Cash games rake for Bucket '+i+ ' users')
fig.show()

In [ ]:
i = '3c'

a = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[0:10])
fig = px.bar(x = a.index,y = a['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Cash Hands',title = 'Cash hands for Bucket '+i+ ' users')
fig.show()

b = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','f_big_blind']],on = 'f_player_id',how = 'left').groupby('days_from_register')['f_big_blind'].mean()[0:10])
fig = px.bar(x = b.index,y = b['f_big_blind'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Average Big Blind',title = 'Average Big Blinds for Bucket '+i+ ' users')
fig.show()

c = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_games[['f_player_id','days_from_register','buyin']],on = 'f_player_id',how = 'left').groupby('days_from_register')['buyin'].mean()[1:11])
fig = px.bar(x = c.index,y = c['buyin'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Buyin',title = 'Tournament Buyin for Bucket '+i+ ' users')
fig.show()

d = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],tourney_hands[['f_player_id','days_from_register','hands']],on = 'f_player_id',how = 'left').groupby('days_from_register')['hands'].mean()[:10])
fig = px.bar(x = d.index,y = d['hands'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Tourney Hands')
fig.show()

e = pd.DataFrame(pd.merge(final.loc[final['bucket']==i,'f_player_id'],cash_games[['f_player_id','days_from_register','rake']],on = 'f_player_id',how = 'left').groupby('days_from_register')['rake'].mean()[:10])
fig = px.bar(x = e.index,y = e['rake'])
fig.update_layout(xaxis_title = 'Days from Register',yaxis_title= 'Rake',title = 'Cash games rake for Bucket '+i+ ' users')
fig.show()

In [ ]:
user3 = pd.merge(user_points,users2[['user','player_id']],on = 'user',how = 'right')

In [ ]:
z = pd.DataFrame(user3.loc[(user3['currentTier']=='Gold') | (user3['currentTier']=='Platinum') | (user3['currentTier']=='Uranium'),'player_id'])

In [ ]:
p = wins.groupby('f_player_id')[['wins','count']].sum()
p['perc'] = p['wins']/p['count']
p.reset_index(inplace=True)
p

In [ ]:
i = '3t'
pd.merge(final[['f_player_id','bucket']].loc[final['bucket']==i],p[['f_player_id','perc']],on = 'f_player_id',how = 'inner')['perc'].mean()

In [ ]:
# %wins:  1- 26.56%   2c - 29.2%  2t - 31.55% 3c - 29.26%  3t - 27.07%

# %flush: 1-55.87%  2c-59.79%  2t- 63.42%  3c- 58.34%  3t- 55.57%

In [ ]:
q = flush.groupby('f_player_id')[['wins','count']].sum()
q['perc'] = q['wins']/q['count']
q.reset_index(inplace=True)
q

In [ ]:
i = '3t'
pd.merge(final[['f_player_id','bucket']].loc[final['bucket']==i],q[['f_player_id','perc']],on = 'f_player_id',how = 'inner')['perc'].mean()